<a href="https://colab.research.google.com/github/npnavas/MAT_421/blob/main/MAT_425_HW_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 19.1 Root Finding Problem
In mathematics it is useful to know when given a function $f(x)$ when that function is zero. In other words we want to know some $x_r$ such that $f(x_r)=0$. This is what is caled root finding. For some functions like $f(x) = x^2-1$ this process is trivial analytically however with a more complex function like $f(x) = J_0(x)$, it's better to have an approximate zero. We'll be using these two functions throughout, here we'll use scipy to find these zeros.

In [12]:
import numpy as np
from scipy import optimize
import scipy.special as spp
# x^2 -1 = 0 -> x = +-1
f1 = lambda x: (x**2)-1
r1 = optimize.fsolve(f1, 0.5)
print("r =", r1)

# Verify the solution is a root
result1 = f1(r1)
print("result=", result1)

# J_0(x) = 0 about 2.4048
f2 = lambda x: spp.j0(x)
r2 = optimize.fsolve(f2, 2)
print("r =", r2)

# Verify the solution is a root
result2 = f2(r2)
print("result=", result2)

r = [1.]
result= [4.4408921e-16]
r = [2.40482556]
result= [9.58688255e-17]


# 19.2 Tolerance
As you might've seen in the last section of code when plugging the approximate zero back into the function, our value wasn't exactly zero. This is where the idea of tolerance comes from. The idea is that there is a "close enough" value within some error of the exact zero we are will to accept. Here we'll talk about two types of tolerance. First since we want $x_r$ close to zero we can say that we can measure error by using $\left|f(x_r)\right|$ as if this value is small we can infer that we have a zero. The second method of error measure is to compare the difference of $x_{i+1}$ and $x_i$ in an iterative method, in other words $|x_{i+1}-x_i|$. This is a useful method since we expect improvments between each iteration so once the difference between two subsequent iteration becomes negliable we get that we'll be close to a root.

# 19.3 Bisection Method
Here we will recall that for $\forall a,b\in\mathbb{R}$ such that $a < b$ for some function $f:[a,b]\to\mathbb{R},\ f(x)$ that if $\text{sign}(f(a))\neq\text{sign}(f(b))$ then $\exists c\in\mathbb{R}$ such that $a < c < b$ where $f(c)=0$. Bisection method takes advantage of this property (called the intermediate value theorem). Bisection method starts off by picking (W.O.L.G.), $a < b$ such that $f(a) > 0$ and $f(b) < 0$. Next you want to find the midpoint $x_m = \frac{a+b}{2}$ and we'll do one of three things


*   If we are close enough to zero or $f(x_m)=0$ then we stop as we've found the root
*   If $f(x_m) > 0$ then $a = x_m$ 
*   If $f(x_m) < 0$ then $b = x_m$

Here we'll write some code to find the first non-negative root of the two functions described in 19.1.

In [29]:
import numpy as np
import scipy.special as spp

def bisect(f, a, b, tol):
  if np.sign(f(a))==np.sign(f(b)): # No root contained in interval
    raise Exception("a and b do not contain a root, interval adjusting to be implimented later")
  
  xm = (a+b)/2
  if np.abs(f(xm)) < tol: # Found root close enough
    return xm 
  elif np.sign(f(a)) == np.sign(f(xm)): # adjust left bound
    return bisect(f, xm, b, tol)
  elif np.sign(f(b)) == np.sign(f(xm)): # adjust right bound
    return bisect(f, a, xm, tol)

f = lambda x: x**2 - 1
r1 = bisect(f, 0, 2, np.finfo(float).eps)
print("root for x^2-1 =", r1)

g = lambda x: spp.j0(x)

r2 = bisect(g, 2, 2.5, np.finfo(float).eps)
print("Root for J0(x) = ", r2)

root for x^2-1 = 1.0
Root for J0(x) =  2.404825557695773


There is a way to ensure that your interval contains 0. Given some aribtary a,b we can do the following loop to ensure that a root is within the interval

```
xm = (a+b)/2
while f(a)*f(b) > 0:
    a = 2*(a-xm) + xm
    b = 2*(b-xm) + xm
```

# 19.4 Newton-Raphson Method
Next let's consider we have a smooth function $f(x)$ who's first derivative is never zero. Taking the first two terms in $f$'s Taylor expansion about some $x_0$ we get
$$ f(x) \approx f(x_0)+f'(x_0)(x-x_0).$$
Let's take some guess $x = x_1$ and solving for $x_1$ we get
$$x_1 = x_0-\dfrac{f(x_0)}{f'(x_0)}.$$
We can keep repeating this process to get 
$$x_i =x_{i-1} \dfrac{f(x_{i-1})}{f'(x_{i-1})},$$
and we can stop once a tolerance condition is met

Let's see this in action using python on the functions from 19.1.

In [36]:
import numpy as np
import scipy.special as spp

def newton(f, df, x0, tol):
  if abs(f(x0)) < tol:
    return x0
  else:
    xn = x0 - (f(x0)/df(x0))
    return newton(f, df, xn, tol)

f = lambda x: x**2 -1
df = lambda x: 2*x
xr1 = newton(f, df, 0.5, np.finfo(float).eps)
print("Root for x^2-1 using Newton-Raphson: ", xr1)

g = lambda x: spp.j0(x)
dg = lambda x: spp.jvp(0, x, n=1)
xr2 = newton(g, dg, 2, np.finfo(float).eps)
print("Root for J0(x) using Newton-Raphson: ", xr2)

Root for x^2-1 using Newton-Raphson:  1.0
Root for J0(x) using Newton-Raphson:  2.4048255576957724


# 19.5 Root Finding In Python
As we saw in 19.1 we saw that scipy has a root finding method `f_solve`. Similar to the method in 19.4 we need to take an initial guess but the `f_solve` method does allow for functions with multiple zeros. Let's use it to find both zeroes of our polynomial and first few zeros of our Bessel function.

In [43]:
import numpy as np
import scipy.special as spp
from scipy.optimize import fsolve

f = lambda x: x**2 - 1
xr1 = fsolve(f, [-2,2])
print("Roots of x^2-1 using f_solve: ", xr1)

g = lambda x: spp.j0(x)
xr2 = fsolve(g, [2,6,9])
print("First three roots of J0(x) using f_solve: ", xr2)
print("Built in zeros of J0(x) for comparison  : ", spp.jn_zeros(0, 3))

Roots of x^2-1 using f_solve:  [-1.  1.]
First three roots of J0(x) using f_solve:  [2.40482556 5.52007811 8.65372791]
Built in zeros of J0(x) for comparison  :  [2.40482556 5.52007811 8.65372791]
